In [ ]:
pip install dart_fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.4.0
    Uninstalling termcolor-2.4.0:
      Successfully uninstalled termcolor-2.4.0


In [ ]:
#import OpenDartReader
import dart_fss as dart
my_api = "c47ac0aef82b83f0b3dbde1aacca2edff70430e5"

dart.set_api_key(api_key = my_api)

ModuleNotFoundError: No module named 'dart_fss'

In [ ]:
import pandas as pd

#상장된 회사 리스트 얻기
all = dart.api.filings.get_corp_code()
df = pd.DataFrame(all)
df_listed = df[df['stock_code'].notnull()]
df_listed.count()

In [ ]:
df_listed

,corp_code,corp_name,stock_code,modify_date
1949,00260985,한빛네트,036720,20170630
1961,00264529,엔플렉스,040130,20170630
1962,00358545,동서정보기술,055000,20170630
2709,00231567,애드모바일,032600,20170630
3785,00359614,리더컴,056140,20170630
...,...,...,...,...
107390,00317104,라이온켐텍,171120,20240401
107391,00651901,에어부산,298690,20240401
107392,00661847,화인베스틸,133820,20240401
107393,01428203,케이씨씨글라스,344820,20240401


##필요한 데이터
* 연결재무제표x, 사업보고서(연간)
* 재무상태표(부채총계, 자본총계, 유동자산, 유동부채, 이익잉여금, 자본금, 자본잉여금)/ 손익계산서(매출총이익, 매출액, 당기순이익)/ 연결재무제표(배당수익률, 배당성향, 배당연속지급)
* 주가 관련 (VHF, 등락률, 베타값)

매출액 = 매출총이익 + 매출원가

In [ ]:
# 필요한 재무 항목
financial_items = [
    'total liabilities', 'total capital', 'current assets', 'current liabilities',
    'retained earnings', 'capital', 'capital surplus', 'gross profit',
    'net profit', 'cost of sales'
]

# 재무 항목에 대응하는 account_nm 값
account_nm_mapping = {
    'total liabilities': '부채총계',
    'total capital': '자본총계',
    'current assets': '유동자산',
    'current liabilities': '유동부채',
    'retained earnings': ('이익잉여금', '이익잉여금(결손금)'),
    'capital': '자본금',
    'capital surplus': ('자본잉여금', '주식발행초과금'),
    'gross profit': ('매출총이익', '매출총이익(손실)'),
    'net profit': ('당기순이익','당기순이익(손실)'),
    'cost of sales': '매출원가',
    'sales': ('매출액', '수익(매출액)'),
}

# 결과를 저장할 데이터프레임 초기화
result_df = pd.DataFrame(columns=['corp_name', 'corp_code', 'stock_code'] + financial_items)

# 각 회사에 대해 재무 데이터 불러오기
for index, row in df_listed.iterrows():
    corp_name = row['corp_name']
    corp_code = row['corp_code']
    stock_code = row['stock_code']
    print(corp_code)

    try:
        # 재무 데이터를 불러오기 (사업보고서, 개별 재무제표 사용)
        fin_data = dart.api.finance.fnltt_singl_acnt_all(corp_code, '2023', reprt_code='11011', fs_div='OFS')
        #OFS:개별재무제표, CFS : 연결재무제표, 1분기 : 11013 반기 : 11012 3분기 : 11014 사업보고서 : 11011

        fin_data = pd.DataFrame(fin_data['list'])

        # 새로운 데이터프레임에 기본 정보 추가
        company_data = {'corp_name': corp_name, 'corp_code': corp_code, 'stock_code': stock_code}

        # 필요한 재무 항목 값 추출
        for item_en, item_ko in account_nm_mapping.items():
            if isinstance(item_ko, tuple):
                # 여러 값 중 하나와 일치하는 경우
                mask = fin_data['account_nm'].isin(item_ko)
            else:
                # 단일 값과 일치하는 경우
                mask = fin_data['account_nm'] == item_ko

            value = fin_data.loc[mask, 'thstrm_amount'].values
            company_data[item_en] = value[0] if len(value) > 0 else None

            # 이전 기간 값 추출 ('net profit'과 'sales'의 경우)
            if item_en == 'net profit':
                previous_values = fin_data.loc[mask, 'bfefrmtrm_amount'].values
                company_data['profit_before'] = previous_values[0] if len(previous_values) > 0 else None

            if item_en == 'sales':
                previous_values = fin_data.loc[mask, 'bfefrmtrm_amount'].values
                company_data['sales_before'] = previous_values[0] if len(previous_values) > 0 else None

        # 결과 데이터프레임에 추가
        result_df = pd.concat([result_df, pd.DataFrame([company_data])], ignore_index=True)

    except Exception as e:
        print(f"Error processing {corp_name}: {e}. Moving to the next company.")
        continue
# 결과 출력
result_df

00260985
Error processing 한빛네트: 조회된 데이타가 없습니다.. Moving to the next company.
00264529
Error processing 엔플렉스: 조회된 데이타가 없습니다.. Moving to the next company.
00358545
Error processing 동서정보기술: 조회된 데이타가 없습니다.. Moving to the next company.
00231567
Error processing 애드모바일: 조회된 데이타가 없습니다.. Moving to the next company.
00359614
Error processing 리더컴: 조회된 데이타가 없습니다.. Moving to the next company.
00153551
Error processing 허메스홀딩스: 조회된 데이타가 없습니다.. Moving to the next company.
00344746
Error processing 유티엑스: 조회된 데이타가 없습니다.. Moving to the next company.
00261188
Error processing 글로포스트: 조회된 데이타가 없습니다.. Moving to the next company.
00268020
Error processing 쏠라엔텍: 조회된 데이타가 없습니다.. Moving to the next company.
00269287
Error processing 보홍: 조회된 데이타가 없습니다.. Moving to the next company.
00475286
Error processing 동북아1호선박투자회사: 조회된 데이타가 없습니다.. Moving to the next company.
00250775
Error processing 에듀아크: 조회된 데이타가 없습니다.. Moving to the next company.
00114321
Error processing 데코: 조회된 데이타가 없습니다.. Moving to the next company.
0019

,corp_name,corp_code,stock_code,total liabilities,total capital,current assets,current liabilities,retained earnings,capital,capital surplus,gross profit,net profit,cost of sales,profit_before,sales,sales_before
0,위노바,00261735,039790,8313681202,2597483955,2438149870,5752347555,-4450545724,4224853562,None,-207562499,-2262962708,6702881851,-9425616040,6495319352,8696868019
1,홈센타홀딩스,00288495,060560,75411748660,113969143560,7527149460,68926715895,-39466590170,63834762500,91409916348,12200020134,5556395226,23605410661,2767452676,35805430795,26714305300
2,우진,00173944,105840,30938927183,148156523469,60575716794,21136890077,49732846721,10161807000,52668984245,27433823847,10510336680,36409362394,-9272613280,63843186241,52107905825
3,인화정공,00482426,101930,128677053936,195939031063,59137786427,120145988837,187398885110,4665852500,6999178683,9772224283,7788111957,62111338670,41570681676,71883562953,54595370405
4,대원산업,00111874,005710,83333433691,404944002637,349162451624,71922433935,394887928385,10018800000,None,60052970267,34667566693,668369463228,29089465023,728422433495,636336318824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,HJ중공업,00633835,097230,2544208000000,341409000000,1724579000000,2122908000000,None,416371000000,1050101000000,-2224000000,-113492000000,2163490000000,-139995000000,2161266000000,1705790000000
2546,라이온켐텍,00317104,171120,34374288995,126779186485,58235338735,34328957495,93758196097,18422228000,14788171873,18816293287,4987742388,79607761760,8519754064,98424055047,150416279224
2547,에어부산,00651901,298690,1171269889193,186813445077,248452066192,361362135670,-272580348780,116640000000,341837250884,213883524974,104145078768,676556823634,-265960609370,None,None
2548,화인베스틸,00661847,133820,119168711542,63467579376,64638053655,115884382073,-14204511202,16234059000,61460615704,-2470551605,-15004464766,132198096829,2374205660,129727545224,124076835252


In [ ]:
result_df.to_csv('financial_data.csv', index=False)

In [ ]:
result_df = pd.read_csv('financial_data.csv')

In [ ]:
result_df

,corp_name,corp_code,stock_code,total liabilities,total capital,current assets,current liabilities,retained earnings,capital,capital surplus,gross profit,net profit,cost of sales,profit_before,sales,sales_before
0,위노바,261735,39790,8.313681e+09,2.597484e+09,2.438150e+09,5.752348e+09,-4.450546e+09,4.224854e+09,NaN,-2.075625e+08,-2.262963e+09,6.702882e+09,-9.425616e+09,6.495319e+09,8.696868e+09
1,홈센타홀딩스,288495,60560,7.541175e+10,1.139691e+11,7.527149e+09,6.892672e+10,-3.946659e+10,6.383476e+10,9.140992e+10,1.220002e+10,5.556395e+09,2.360541e+10,2.767453e+09,3.580543e+10,2.671431e+10
2,우진,173944,105840,3.093893e+10,1.481565e+11,6.057572e+10,2.113689e+10,4.973285e+10,1.016181e+10,5.266898e+10,2.743382e+10,1.051034e+10,3.640936e+10,-9.272613e+09,6.384319e+10,5.210791e+10
3,인화정공,482426,101930,1.286771e+11,1.959390e+11,5.913779e+10,1.201460e+11,1.873989e+11,4.665852e+09,6.999179e+09,9.772224e+09,7.788112e+09,6.211134e+10,4.157068e+10,7.188356e+10,5.459537e+10
4,대원산업,111874,5710,8.333343e+10,4.049440e+11,3.491625e+11,7.192243e+10,3.948879e+11,1.001880e+10,NaN,6.005297e+10,3.466757e+10,6.683695e+11,2.908947e+10,7.284224e+11,6.363363e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,HJ중공업,633835,97230,2.544208e+12,3.414090e+11,1.724579e+12,2.122908e+12,NaN,4.163710e+11,1.050101e+12,-2.224000e+09,-1.134920e+11,2.163490e+12,-1.399950e+11,2.161266e+12,1.705790e+12
2546,라이온켐텍,317104,171120,3.437429e+10,1.267792e+11,5.823534e+10,3.432896e+10,9.375820e+10,1.842223e+10,1.478817e+10,1.881629e+10,4.987742e+09,7.960776e+10,8.519754e+09,9.842406e+10,1.504163e+11
2547,에어부산,651901,298690,1.171270e+12,1.868134e+11,2.484521e+11,3.613621e+11,-2.725803e+11,1.166400e+11,3.418373e+11,2.138835e+11,1.041451e+11,6.765568e+11,-2.659606e+11,NaN,NaN
2548,화인베스틸,661847,133820,1.191687e+11,6.346758e+10,6.463805e+10,1.158844e+11,-1.420451e+10,1.623406e+10,6.146062e+10,-2.470552e+09,-1.500446e+10,1.321981e+11,2.374206e+09,1.297275e+11,1.240768e+11


In [ ]:
result_df.isnull().sum()

,0
corp_name,0
corp_code,0
stock_code,0
total liabilities,17
total capital,20
current assets,109
current liabilities,112
retained earnings,84
capital,72
capital surplus,473


In [ ]:
result_df.drop( columns = ['cost of sales'], inplace=True)

In [ ]:
fin_1 = result_df.dropna(axis=0)
len(fin_1)

1573

### 재무 안정성
1. 부채비율 = (부채총계 / 자본총계) × 100
2. 유동비율 = (유동자산 / 유동부채) × 100
3. 유보율(단위%) = ((이익잉여금+자본잉여금) ÷ 자본금) × 100

### 수익성
1. 매출총이익률 = 매출총이익 / 매출액
2. 순이익률 = (당기순이익 / 매출액) *100(%)
3. ROE = 당기순이익 ÷ 자본총계 × 100

### 성장성
1. 연평균 매출 성장률 = (최종 년도의 매출액/최초 년도의 매출액)^(1/기간)-1 -> **3년간**
2. 연평균 순이익 증감률 = (최종 년도의 순이익/ 최초 년도의 순이익)^(1/기간)-1 -> **3년간**

In [ ]:
fin_1  #결측치 제거한 df

,corp_name,corp_code,stock_code,total liabilities,total capital,current assets,current liabilities,retained earnings,capital,capital surplus,...,sales,sales_before,liabilities_ratio,current_ratio,retention_rate,gross_profit_rate,net_profit_rate,ROE,sales_growth_rate,profit_change_rate
1,홈센타홀딩스,288495,060560,7.541175e+10,1.139691e+11,7.527149e+09,6.892672e+10,-3.946659e+10,6.383476e+10,9.140992e+10,...,3.580543e+10,2.671431e+10,66.168567,10.920511,81.371535,0.340731,15.518303,4.875351,0.102559,0.261550
2,우진,173944,105840,3.093893e+10,1.481565e+11,6.057572e+10,2.113689e+10,4.973285e+10,1.016181e+10,5.266898e+10,...,6.384319e+10,5.210791e+10,20.882595,286.587651,1007.712811,0.429706,16.462738,7.094076,0.070049,-1.000000
3,인화정공,482426,101930,1.286771e+11,1.959390e+11,5.913779e+10,1.201460e+11,1.873989e+11,4.665852e+09,6.999179e+09,...,7.188356e+10,5.459537e+10,65.671986,49.221607,4166.399684,0.135945,10.834343,3.974763,0.096035,-0.427799
5,대동,109286,000490,7.044586e+11,3.087462e+11,5.966778e+11,5.796233e+11,1.831844e+11,2.539259e+10,4.218304e+10,...,1.032870e+12,8.861147e+11,228.167513,102.942341,887.532262,0.154375,0.876244,2.931361,0.052411,-0.174628
6,삼화콘덴서공업,129350,001820,5.364885e+10,2.172845e+11,1.413673e+11,4.769648e+10,1.916191e+11,1.039500e+10,5.376571e+09,...,2.249395e+11,2.113443e+11,24.690597,296.389492,1895.100721,0.178229,8.309545,8.602293,0.020998,-0.110413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,비상교육,520887,100220,1.786740e+11,1.733318e+11,1.328325e+11,1.763874e+11,1.279872e+11,6.498370e+09,5.489121e+10,...,2.383168e+11,1.677416e+11,103.082075,75.307216,2814.219079,0.599380,-8.477327,-11.655624,0.124186,-1.000000
2542,누리플렉스,269889,040160,5.427438e+10,7.706953e+10,6.507374e+10,4.868984e+10,3.839774e+10,6.027768e+09,3.257401e+10,...,7.725673e+10,3.236482e+10,70.422621,133.649525,1177.413636,0.259896,1.274460,1.277556,0.336455,-0.378828
2546,라이온켐텍,317104,171120,3.437429e+10,1.267792e+11,5.823534e+10,3.432896e+10,9.375820e+10,1.842223e+10,1.478817e+10,...,9.842406e+10,1.504163e+11,27.113511,169.639112,589.214117,0.191176,5.067605,3.934197,-0.131835,-0.163449
2548,화인베스틸,661847,133820,1.191687e+11,6.346758e+10,6.463805e+10,1.158844e+11,-1.420451e+10,1.623406e+10,6.146062e+10,...,1.297275e+11,1.240768e+11,187.763127,55.778054,291.092354,-0.019044,-11.566136,-23.641149,0.014956,-1.000000


In [ ]:
import numpy as np

fin_1.replace('', np.nan, inplace=True)
fin_2 = fin_1.drop(columns=['corp_name', 'corp_code', 'stock_code']).astype(float)
fin_1 = pd.concat([fin_1[['corp_name', 'corp_code', 'stock_code']], fin_2], axis=1)
fin_1.dtypes

<ipython-input-14-2d1299ce7780>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_1.replace('', np.nan, inplace=True)


,0
corp_name,object
corp_code,int64
stock_code,int64
total liabilities,float64
total capital,float64
current assets,float64
current liabilities,float64
retained earnings,float64
capital,float64
capital surplus,float64


In [ ]:
# 재무안정성 지표 계산
# 1)부채비율
fin_1['liabilities_ratio'] = (fin_1['total liabilities'] / fin_1['total capital'])*100

# 2)유동비율
fin_1['current_ratio'] = (fin_1['current assets'] / fin_1['current liabilities']) * 100

# 3)유보율
fin_1['retention_rate'] = ((fin_1['retained earnings'] + fin_1['capital surplus']) / fin_1['capital']) * 100

# 수익성 지표 계산
# 4)매출총이익률
fin_1['gross_profit_rate'] = fin_1['gross profit'] / fin_1['sales']
fin_1.loc[fin_1['sales'] == 0, 'gross_profit_rate'] = 0

# 5)순이익률
fin_1['net_profit_rate'] = (fin_1['net profit'] / fin_1['sales']) * 100

# 6)ROE
fin_1['ROE'] = (fin_1['net profit'] / fin_1['total capital']) * 100

#성장성 지표 계산
# 7)연평균 매출 성장률
fin_1['sales_growth_rate'] = np.where(
    (fin_1['sales'] > 0) & (fin_1['sales_before'] > 0),
    ((fin_1['sales'] / fin_1['sales_before']) ** (1/3)) - 1,
    -1
)

# 8)연평균 순이익 증감률
fin_1['profit_change_rate'] = np.where(
    (fin_1['net profit'] > 0) & (fin_1['profit_before'] > 0),
    ((fin_1['net profit'] / fin_1['profit_before']) ** (1/3)) - 1,
    -1
)



---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd

In [ ]:
divide_df = pd.read_csv('divide.csv')

In [ ]:
divide_df.columns = divide_df.iloc[0]
divide_df = divide_df.drop(index=0)
divide_df.reset_index(drop=True, inplace=True)

divide_df

,corp_name,stock_code,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
0,3S,060310,주당액면가액(원),500,500,500
1,3S,060310,(연결)당기순이익(백만원),"1,431","1,171",695
2,3S,060310,(별도)당기순이익(백만원),"2,097",482,754
3,3S,060310,(연결)주당순이익(원),30,33,16
4,3S,060310,현금배당금총액(백만원),-,-,-
...,...,...,...,...,...,...
49023,힘스,238490,주식배당수익률(%),-,-,-
49024,힘스,238490,주당 현금배당금(원),50,-,100
49025,힘스,238490,주당 현금배당금(원),-,-,-
49026,힘스,238490,주당 주식배당(주),-,-,-


In [ ]:
divide_df.dtypes

,0
0,
corp_name,object
stock_code,object
account_nm,object
thstrm_amount,object
frmtrm_amount,object
bfefrmtrm_amount,object


In [ ]:
# stock_code 리스트 추출
fin_1['stock_code'] = fin_1['stock_code'].astype(str).str.zfill(6)  #6자리 맞춰주기
stock_codes = fin_1['stock_code'].tolist()
len(stock_codes)

1573

In [ ]:
# Prepare a DataFrame to store results
columns = ['stock_code', 'cashdivide', 'dividend', 'divide_count']
result_df1 = pd.DataFrame(columns=columns)

for stock_code in stock_codes:
    presence_code = 0
    cashdivide = None
    dividend = None

    # Filter rows for the current stock_code
    filtered_df = divide_df[divide_df['stock_code'] == stock_code]

    for index, row in filtered_df.iterrows():
        if row['account_nm'] == '현금배당수익률(%)':
            if row['thstrm_amount'] != '-':
                presence_code = 1
                value = row['thstrm_amount']

            if row['thstrm_amount'] != '-' and row['frmtrm_amount'] != '-':
                presence_code = 2

            if row['thstrm_amount'] != '-' and row['frmtrm_amount'] != '-' and row['bfefrmtrm_amount'] != '-':
                presence_code = 3

        elif row['account_nm'] == '(연결)현금배당성향(%)':
            if pd.notna(row['thstrm_amount']) and row['thstrm_amount'] != '-':
                value1 = row['thstrm_amount']

    new_row = pd.DataFrame({
        'stock_code': [row['stock_code']],
        'cashdivide': [value],
        'dividend': [value1],
        'divide_count': [presence_code]
    })

    # Concatenate the new row to result_df1
    result_df1 = pd.concat([result_df1, new_row], ignore_index=True)

result_df1

,stock_code,cashdivide,dividend,divide_count
0,060560,0.80,8.10,2
1,105840,1.70,33.90,3
2,101930,19.10,817.60,1
3,000490,0.60,21.00,3
4,001820,1.30,24.70,3
...,...,...,...,...
1568,100220,3.70,80.71,0
1569,040160,0.90,19.20,1
1570,171120,3.00,50.61,3
1571,133820,3.00,50.61,0


In [ ]:
fin_2 = result_df1.dropna(axis=0)
len(fin_2)

1573

In [ ]:
# 최종
merged_df = pd.merge(fin_1, fin_2, on='stock_code', how='inner')
merged_df

,corp_name,corp_code,stock_code,total liabilities,total capital,current assets,current liabilities,retained earnings,capital,capital surplus,...,current_ratio,retention_rate,gross_profit_rate,net_profit_rate,ROE,sales_growth_rate,profit_change_rate,cashdivide,dividend,divide_count
0,홈센타홀딩스,288495,060560,7.541175e+10,1.139691e+11,7.527149e+09,6.892672e+10,-3.946659e+10,6.383476e+10,9.140992e+10,...,10.920511,81.371535,0.340731,15.518303,4.875351,0.102559,0.261550,0.80,8.10,2
1,우진,173944,105840,3.093893e+10,1.481565e+11,6.057572e+10,2.113689e+10,4.973285e+10,1.016181e+10,5.266898e+10,...,286.587651,1007.712811,0.429706,16.462738,7.094076,0.070049,-1.000000,1.70,33.90,3
2,인화정공,482426,101930,1.286771e+11,1.959390e+11,5.913779e+10,1.201460e+11,1.873989e+11,4.665852e+09,6.999179e+09,...,49.221607,4166.399684,0.135945,10.834343,3.974763,0.096035,-0.427799,19.10,817.60,1
3,대동,109286,000490,7.044586e+11,3.087462e+11,5.966778e+11,5.796233e+11,1.831844e+11,2.539259e+10,4.218304e+10,...,102.942341,887.532262,0.154375,0.876244,2.931361,0.052411,-0.174628,0.60,21.00,3
4,삼화콘덴서공업,129350,001820,5.364885e+10,2.172845e+11,1.413673e+11,4.769648e+10,1.916191e+11,1.039500e+10,5.376571e+09,...,296.389492,1895.100721,0.178229,8.309545,8.602293,0.020998,-0.110413,1.30,24.70,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,비상교육,520887,100220,1.786740e+11,1.733318e+11,1.328325e+11,1.763874e+11,1.279872e+11,6.498370e+09,5.489121e+10,...,75.307216,2814.219079,0.599380,-8.477327,-11.655624,0.124186,-1.000000,3.70,80.71,0
1569,누리플렉스,269889,040160,5.427438e+10,7.706953e+10,6.507374e+10,4.868984e+10,3.839774e+10,6.027768e+09,3.257401e+10,...,133.649525,1177.413636,0.259896,1.274460,1.277556,0.336455,-0.378828,0.90,19.20,1
1570,라이온켐텍,317104,171120,3.437429e+10,1.267792e+11,5.823534e+10,3.432896e+10,9.375820e+10,1.842223e+10,1.478817e+10,...,169.639112,589.214117,0.191176,5.067605,3.934197,-0.131835,-0.163449,3.00,50.61,3
1571,화인베스틸,661847,133820,1.191687e+11,6.346758e+10,6.463805e+10,1.158844e+11,-1.420451e+10,1.623406e+10,6.146062e+10,...,55.778054,291.092354,-0.019044,-11.566136,-23.641149,0.014956,-1.000000,3.00,50.61,0


In [ ]:
merged_df.to_csv("total_fin.csv")

In [ ]:
merged_df = pd.read_csv('total_fin.csv', index_col=0)

In [ ]:
merged_df1 = merged_df.drop(columns=['total liabilities', 'total capital', 'current assets', 'current liabilities', 'retained earnings', 'capital', 'capital surplus', 'gross profit', 'net profit', 'profit_before',
       'sales', 'sales_before'], axis=1)
merged_df1

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,corp_name,corp_code,stock_code,liabilities_ratio,current_ratio,retention_rate,gross_profit_rate,net_profit_rate,ROE,sales_growth_rate,profit_change_rate,cashdivide,dividend,divide_count
0,홈센타홀딩스,288495,60560,66.168567,10.920511,81.371535,0.340731,15.518303,4.875351,0.102559,0.261550,0.80,8.10,2
1,우진,173944,105840,20.882595,286.587651,1007.712811,0.429706,16.462738,7.094076,0.070049,-1.000000,1.70,33.90,3
2,인화정공,482426,101930,65.671986,49.221607,4166.399684,0.135945,10.834343,3.974763,0.096035,-0.427799,19.10,817.60,1
3,대동,109286,490,228.167513,102.942341,887.532262,0.154375,0.876244,2.931361,0.052411,-0.174628,0.60,21.00,3
4,삼화콘덴서공업,129350,1820,24.690597,296.389492,1895.100721,0.178229,8.309545,8.602293,0.020998,-0.110413,1.30,24.70,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,비상교육,520887,100220,103.082075,75.307216,2814.219079,0.599380,-8.477327,-11.655624,0.124186,-1.000000,3.70,80.71,0
1569,누리플렉스,269889,40160,70.422621,133.649525,1177.413636,0.259896,1.274460,1.277556,0.336455,-0.378828,0.90,19.20,1
1570,라이온켐텍,317104,171120,27.113511,169.639112,589.214117,0.191176,5.067605,3.934197,-0.131835,-0.163449,3.00,50.61,3
1571,화인베스틸,661847,133820,187.763127,55.778054,291.092354,-0.019044,-11.566136,-23.641149,0.014956,-1.000000,3.00,50.61,0


In [ ]:
#stock_code type 변경
merged_df1['stock_code'] = merged_df1['stock_code'].astype(str).str.zfill(6)

# stock_code 리스트 추출
stock_codes = merged_df1['stock_code'].tolist()

변동성 기준 데이터

* VHF
* 등락률
* 베타값

In [ ]:
#VHF
VHF = pd.read_csv('VHF.csv')
VHF.rename(columns={'ticker': 'stock_code'}, inplace=True)

#등락률
updown = pd.read_csv('updown.csv')

#베타값
beta = pd.read_csv('beta.csv')

In [ ]:
# Merge VHF data
merged_df1 = pd.merge(merged_df1, VHF, on='stock_code', how='left')

# Merge updown data
merged_df1 = pd.merge(merged_df1, updown, on='stock_code', how='left')

# Merge beta data
merged_df1 = pd.merge(merged_df1, beta, on='stock_code', how='left')
merged_df1

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,corp_name,corp_code,stock_code,liabilities_ratio,current_ratio,retention_rate,gross_profit_rate,net_profit_rate,ROE,sales_growth_rate,profit_change_rate,cashdivide,dividend,divide_count,VHF,updown,beta
0,홈센타홀딩스,288495,060560,66.168567,10.920511,81.371535,0.340731,15.518303,4.875351,0.102559,0.261550,0.80,8.10,2,38.709677,5.99,0.60
1,우진,173944,105840,20.882595,286.587651,1007.712811,0.429706,16.462738,7.094076,0.070049,-1.000000,1.70,33.90,3,28.333333,10.98,0.80
2,인화정공,482426,101930,65.671986,49.221607,4166.399684,0.135945,10.834343,3.974763,0.096035,-0.427799,19.10,817.60,1,37.647059,-7.62,0.59
3,대동,109286,000490,228.167513,102.942341,887.532262,0.154375,0.876244,2.931361,0.052411,-0.174628,0.60,21.00,3,63.759214,31.43,0.93
4,삼화콘덴서공업,129350,001820,24.690597,296.389492,1895.100721,0.178229,8.309545,8.602293,0.020998,-0.110413,1.30,24.70,3,18.009479,27.00,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,비상교육,520887,100220,103.082075,75.307216,2814.219079,0.599380,-8.477327,-11.655624,0.124186,-1.000000,3.70,80.71,0,38.000000,-7.35,0.47
1569,누리플렉스,269889,040160,70.422621,133.649525,1177.413636,0.259896,1.274460,1.277556,0.336455,-0.378828,0.90,19.20,1,77.952756,-14.29,0.48
1570,라이온켐텍,317104,171120,27.113511,169.639112,589.214117,0.191176,5.067605,3.934197,-0.131835,-0.163449,3.00,50.61,3,34.666667,-4.14,0.43
1571,화인베스틸,661847,133820,187.763127,55.778054,291.092354,-0.019044,-11.566136,-23.641149,0.014956,-1.000000,3.00,50.61,0,28.644501,-39.57,1.42


In [ ]:
merged_df1.to_csv("final_data.csv")

In [ ]:
merged_df1.isnull().sum()

,0
corp_name,0
corp_code,0
stock_code,0
liabilities_ratio,0
current_ratio,0
retention_rate,0
gross_profit_rate,0
net_profit_rate,0
ROE,0
sales_growth_rate,0


In [ ]:
a = merged_df1.dropna(axis=0)
len(a)

1417

In [ ]:
a.to_csv("final_data.csv")



---



In [ ]:
corp_code = '01428203'
data = dart.api.finance.fnltt_singl_acnt_all(corp_code, '2023', '11011', 'CFS')

data = pd.DataFrame(data['list'])
data

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20240328001643,11011,2023,01428203,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 4 기,2223679597409,제 3 기,2209564785783,제 2 기,1877935028583,7,KRW,NaN
1,20240328001643,11011,2023,01428203,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 4 기,912921220598,제 3 기,958080961323,제 2 기,792918296257,8,KRW,NaN
2,20240328001643,11011,2023,01428203,BS,재무상태표,dart_ShortTermAdvancePayments,유동선급금,-,제 4 기,79349317633,제 3 기,60953170625,제 2 기,9257789042,9,KRW,NaN
3,20240328001643,11011,2023,01428203,BS,재무상태표,dart_ShortTermPrepaidExpenses,선급비용,-,제 4 기,1447182981,제 3 기,1642391135,제 2 기,883688837,10,KRW,NaN
4,20240328001643,11011,2023,01428203,BS,재무상태표,-표준계정코드 미사용-,반환제품회수권,-,제 4 기,607572587,제 3 기,300520618,제 2 기,70526511,11,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,20240328001643,11011,2023,01428203,SCE,자본변동표,ifrs-full_ShareOfOtherComprehensiveIncomeOfAss...,지분법자본변동,자본 [구성요소]|지배기업의 소유주에게 귀속되는 지분 [구성요소]|자본금 [구성요소],제 4 기,0,제 3 기,0,제 2 기,0,13,KRW,NaN
205,20240328001643,11011,2023,01428203,SCE,자본변동표,ifrs-full_ShareOfOtherComprehensiveIncomeOfAss...,지분법자본변동,자본 [구성요소]|지배기업의 소유주에게 귀속되는 지분 [구성요소]|이익잉여금 [구성요소],제 4 기,0,제 3 기,0,제 2 기,0,13,KRW,NaN
206,20240328001643,11011,2023,01428203,SCE,자본변동표,ifrs-full_ShareOfOtherComprehensiveIncomeOfAss...,지분법자본변동,자본 [구성요소]|지배기업의 소유주에게 귀속되는 지분 [구성요소]|기타포괄손익누계액...,제 4 기,-322492772,제 3 기,0,제 2 기,0,13,KRW,NaN
207,20240328001643,11011,2023,01428203,SCE,자본변동표,ifrs-full_ShareOfOtherComprehensiveIncomeOfAss...,지분법자본변동,자본 [구성요소]|지배기업의 소유주에게 귀속되는 지분 [구성요소]|기타자본구성요소 ...,제 4 기,0,제 3 기,0,제 2 기,0,13,KRW,NaN


In [ ]:
data['account_nm'].unique()

array(['자산총계', '유동자산', '유동선급금', '선급비용', '반환제품회수권', '현금및현금성자산', '유동매출채권',
       '재고자산', '기타유동자산', '기타유동금융자산', '기타 유동채권', '비유동자산', '순확정급여자산',
       '장기선급금', '장기선급비용', '기타채권', '무형자산', '관계기업투자', '투자부동산', '비유동매출채권',
       '기타비유동금융자산', '유형자산', '사용권자산', '자본총계', '지배기업의 소유주에게 귀속되는 자본',
       '자본잉여금', '자본조정', '기타포괄손익누계액', '자본금', '이익잉여금(결손금)', '비지배지분',
       '자본과부채총계', '유동부채', '단기매입채무', '기타채무', '유동성사채',
       '장기차입금의 유동성장기부채로의 대체', '유동충당부채', '당기법인세부채', '기타유동금융부채', '기타 유동부채',
       '단기차입금', '부채총계', '비유동부채', '확정급여부채', '이연법인세부채', '장기차입금', '비유동성사채',
       '비유동충당부채', '기타장기종업원급여부채', '기타 비유동 부채', '영업이익(손실)', '기타이익', '기타손실',
       '판매비와관리비', '총포괄손익', '포괄손익, 비지배지분', '포괄손익, 지배기업의 소유주에게 귀속되는 지분',
       '매출원가', '기본주당이익(손실)', '희석주당이익(손실)', '금융원가', '금융수익', '매출총이익',
       '법인세비용(수익)', '기타포괄손익', '당기손익으로 재분류될 수 있는 항목(세후기타포괄손익)',
       '해외사업장환산외환차이(세후기타포괄손익)', '지분법자본변동', '당기손익으로 재분류되지 않는항목(세후기타포괄손익)',
       '확정급여제도의 재측정손익(세후기타포괄손익)', '당기순이익(손실)', '비지배지분에 귀속되는 당기순이익(손실)',
       '지배기업의 소유주에게 귀속되는 당기순이익